In [19]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import yfinance as yf
from pytrends.request import TrendReq

In [20]:
posts = pd.read_csv("trimmed_posts.csv")

In [21]:
selected_cols = ['title','selftext']
ticker_list = ['MSFT','TSLA','AMZN','AAPL','JPM','CRM','ZM','PTON','DIS','MCD','PRTY','CSCO','GOOGL','ORCL','GME','AMC','BB','NOK','BBBY','BBY','KOSS','EXPR']

In [22]:

counter = 0
for ticker in ticker_list:
    new_df = posts[posts[selected_cols].apply(lambda x: x.str.contains(ticker)).all(axis=1)]
    new_df['Ticker'] = ticker
    new_df.to_csv(f'{ticker}Report.csv')
    if counter < 1:
        data = new_df
    else:
        data = data.append(new_df, ignore_index=True)
        
    group_df = new_df.groupby('created_utc') \
       .agg({'id':'size', 'num_comments':'mean', 'score':'mean'}) \
       .rename(columns={'id':'post_count','num_comments':'avg_comments', 'score':'avg_score'}) \
       .reset_index()
    group_df['Ticker'] = ticker
    if counter < 1:
        group_data = group_df
    else:
        group_data = group_data.append(group_df, ignore_index=True)
    counter = counter + 1

<ipython-input-22-4225bda3de48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Ticker'] = ticker


In [23]:
group_data

,created_utc,post_count,avg_comments,avg_score,Ticker
0,2020-12-08,1,4.000000,1.000000,MSFT
1,2020-12-09,3,2.666667,1.000000,MSFT
2,2020-12-10,1,17.000000,1.000000,MSFT
3,2020-12-16,5,40.800000,4.600000,MSFT
4,2020-12-17,4,26.250000,6.250000,MSFT
...,...,...,...,...,...
601,2021-01-30,6,0.000000,1.000000,EXPR
602,2021-01-31,14,0.285714,1.071429,EXPR
603,2021-02-01,10,0.800000,1.000000,EXPR
604,2021-02-02,11,2.363636,2.181818,EXPR


In [24]:

super_posts_df = data.loc[data['score'] > 300]
super_posts_df = super_posts_df.groupby(['created_utc', 'Ticker'])['score'].apply(lambda x: (x>=300).sum()).reset_index(name='Count_Score_300')

super_posts_df

,created_utc,Ticker,Count_Score_300
0,2020-12-17,TSLA,1
1,2020-12-18,GME,2
2,2020-12-18,TSLA,6
3,2021-01-13,GME,1
4,2021-01-14,GME,1
5,2021-01-19,GME,1
6,2021-01-27,AMC,8
7,2021-01-27,BB,8
8,2021-01-27,GME,35
9,2021-01-27,NOK,2


In [25]:
beginDate = '2020-12-01'
endDate = datetime.today().strftime('%Y-%m-%d')

counter = 0
for stock in ticker_list:
    pytrend = TrendReq(hl='en-US', tz=360)
    pytrend.build_payload(kw_list=[stock], timeframe=beginDate + ' ' + endDate, geo='US')
    df = pytrend.interest_over_time()
    pytrend = TrendReq(hl='en-US', tz=360)
    pytrend.build_payload(kw_list=[stock], timeframe=beginDate + ' ' + endDate, geo='US')
    df = pytrend.interest_over_time()
    df['Noise'] = df[stock]
    df[stock]=stock
    df.index.names = ['Date']
    df.columns = ['Stock','isPartial','Noise']
    if counter < 1:
        mergedNoise = df
    else:
        mergedNoise = pd.merge(mergedNoise, df, on=['Date', 'Stock', 'isPartial', 'Noise'], how='outer')  
    counter = counter + 1

In [26]:
beginDate = '2020-12-01'
endDate = '2021-03-01'


counter = 0
for ticker in ticker_list:
    stockData = yf.download(ticker, start=beginDate, end=endDate)
    stockData['Ticker'] = ticker
    stockReport = pd.DataFrame(stockData, columns= ['Ticker','Adj Close','Volume'])
    if counter < 1:
        mergedReport = stockReport
    else:
        mergedReport = pd.merge(mergedReport, stockReport, on=['Date', 'Ticker','Adj Close', 'Volume'], how='outer')
    counter = counter + 1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [27]:
group_data['created_utc'] = pd.to_datetime(group_data['created_utc'])

In [28]:
group_data

,created_utc,post_count,avg_comments,avg_score,Ticker
0,2020-12-08,1,4.000000,1.000000,MSFT
1,2020-12-09,3,2.666667,1.000000,MSFT
2,2020-12-10,1,17.000000,1.000000,MSFT
3,2020-12-16,5,40.800000,4.600000,MSFT
4,2020-12-17,4,26.250000,6.250000,MSFT
...,...,...,...,...,...
601,2021-01-30,6,0.000000,1.000000,EXPR
602,2021-01-31,14,0.285714,1.071429,EXPR
603,2021-02-01,10,0.800000,1.000000,EXPR
604,2021-02-02,11,2.363636,2.181818,EXPR


In [29]:
merged = group_data.merge(mergedNoise, left_on=['created_utc', 'Ticker'], right_on=['Date', 'Stock'], how='left')

In [30]:
merged.drop(columns=['Stock', 'isPartial'])

,created_utc,post_count,avg_comments,avg_score,Ticker,Noise
0,2020-12-08,1,4.000000,1.000000,MSFT,28
1,2020-12-09,3,2.666667,1.000000,MSFT,23
2,2020-12-10,1,17.000000,1.000000,MSFT,29
3,2020-12-16,5,40.800000,4.600000,MSFT,26
4,2020-12-17,4,26.250000,6.250000,MSFT,28
...,...,...,...,...,...,...
601,2021-01-30,6,0.000000,1.000000,EXPR,8
602,2021-01-31,14,0.285714,1.071429,EXPR,10
603,2021-02-01,10,0.800000,1.000000,EXPR,22
604,2021-02-02,11,2.363636,2.181818,EXPR,20


In [32]:
super_posts_df['created_utc'] = pd.to_datetime(super_posts_df['created_utc'])

In [33]:
merged = merged.merge(super_posts_df, left_on=['created_utc', 'Ticker'], right_on=['created_utc', 'Ticker'], how="left")

In [34]:
merged

,created_utc,post_count,avg_comments,avg_score,Ticker,Stock,isPartial,Noise,Count_Score_300
0,2020-12-08,1,4.000000,1.000000,MSFT,MSFT,False,28,NaN
1,2020-12-09,3,2.666667,1.000000,MSFT,MSFT,False,23,NaN
2,2020-12-10,1,17.000000,1.000000,MSFT,MSFT,False,29,NaN
3,2020-12-16,5,40.800000,4.600000,MSFT,MSFT,False,26,NaN
4,2020-12-17,4,26.250000,6.250000,MSFT,MSFT,False,28,NaN
...,...,...,...,...,...,...,...,...,...
601,2021-01-30,6,0.000000,1.000000,EXPR,EXPR,False,8,NaN
602,2021-01-31,14,0.285714,1.071429,EXPR,EXPR,False,10,NaN
603,2021-02-01,10,0.800000,1.000000,EXPR,EXPR,False,22,NaN
604,2021-02-02,11,2.363636,2.181818,EXPR,EXPR,False,20,NaN


In [52]:
mergedReport

,Ticker,Adj Close,Volume
Date,,,
2020-12-01,MSFT,215.713181,30931300
2020-12-02,MSFT,214.875107,23724500
2020-12-03,MSFT,213.747711,25120900
2020-12-04,MSFT,213.867432,24666000
2020-12-07,MSFT,213.797577,24620000
...,...,...,...
2021-02-22,EXPR,2.470000,13881100
2021-02-23,EXPR,2.310000,8924100
2021-02-24,EXPR,3.250000,37326300


In [35]:
merged = merged.merge(mergedReport, left_on=['created_utc', 'Ticker'], right_on=['Date', 'Ticker'], how='left')

In [36]:
merged

,created_utc,post_count,avg_comments,avg_score,Ticker,Stock,isPartial,Noise,Count_Score_300,Adj Close,Volume
0,2020-12-08,1,4.000000,1.000000,MSFT,MSFT,False,28,NaN,215.513626,23284100.0
1,2020-12-09,3,2.666667,1.000000,MSFT,MSFT,False,23,NaN,211.313309,32440600.0
2,2020-12-10,1,17.000000,1.000000,MSFT,MSFT,False,29,NaN,210.036255,26733300.0
3,2020-12-16,5,40.800000,4.600000,MSFT,MSFT,False,26,NaN,218.776123,35023300.0
4,2020-12-17,4,26.250000,6.250000,MSFT,MSFT,False,28,NaN,218.915802,32515800.0
...,...,...,...,...,...,...,...,...,...,...,...
601,2021-01-30,6,0.000000,1.000000,EXPR,EXPR,False,8,NaN,NaN,NaN
602,2021-01-31,14,0.285714,1.071429,EXPR,EXPR,False,10,NaN,NaN,NaN
603,2021-02-01,10,0.800000,1.000000,EXPR,EXPR,False,22,NaN,5.000000,28988900.0
604,2021-02-02,11,2.363636,2.181818,EXPR,EXPR,False,20,NaN,3.380000,35425000.0


In [37]:
merged.to_csv('Subset_Report.csv')